In [1]:
import gym
import tensorflow as tf
import numpy as np

In [3]:
env = gym.make('CartPole-v0')

print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


In [ ]:
state_size = 4
action_size = 2

max_episodes = 1000
learning_rate = 0.001
gamma = 0.99

In [ ]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_epiosde_rewards[i] = cumulative
    
    mean = np.mean(discounted_epiosde_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / std
    
    return discounted_episode_rewards

In [ ]:
class NeuralNetwork():
    def __init__(self, name):
        with tf.name_scope(name):
            inputs = tf.placeholder(tf.float32, [None, state_size], name="inputs")
            actions = tf.placeholder(tf.int32, [None, action_size], name="actions")

            fc1 = tf.layers.dense(inputs, 20, activation=tf.nn.relu, name='fc1')
            fc2 = tf.layers.dense(fc1, 10, activation=tf.nn.relu, name='fc2')
            fc3 = tf.layers.dense(fc2, 2, activation=None, name='fc3')            
            
            output = tf.nn.log_softmax(fc3)
            
            tf.gradients()

with tf.device("/device:GPU:0"):             
    tf.reset_default_graph()
    NN = NeuralNetwork("main")

In [ ]:
save_file = 'REINFORCEMENT.ckpt'
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:    
    sess.run(tf.global_variables_initializer())
    
    for ep in range(max_episodes):
        episode_rewards_sum = 0
        episode_states = []
        episode_actions = []
        episode_rewards = []
        
        state = env.reset()
        
        while True:
            output = sess.run(NN.output, feed_dict={inputs: state.reshape([1, state_size])})
            action = np.random.choice(action_size, p=output)
            
            next_state, reward, done, info = env.step(action)
            
            #action encoding
            actions = np.zeros(action_size)
            actions[action] = 1
            
            episode_states.append(state)            
            episode_actions.append(actions)
            episode_rewards.append(reward)
            
            if done:
                episode_rewards_sum = np.sum(episode_rewards)
                
                allRewards.append(episode_rewards_sum)
                
                total_rewards = np.sum(allRewards)
                
                mean_reward = np.divide(total_rewards, episode+1)
                
                maxRewardRecorded = np.amax(allRewards)
                
                print()
                print()
                print()
                print()
                print()
                
                discounted_episode_rewards = discount_and_normalize_rewards(episode_rewards)
                
                loss_, _ = sess.run([NN.loss, NN.train_opt], feed_dict={})
                break
            
            state = next_state

    
    plt.figure(figsize=(30, 4))
    x = np.array(range(1, train_episodes+1))
    plt.scatter(x, scores, marker='>')        
    plt.xlabel("Episodes")
    plt.ylabel("Scores")
    plt.show()    
    
    save_path = saver.save(sess, save_file)
    print("Model saved in path: %s" % save_path)

In [ ]:
tf.reset_default_graph()

with tf.device("/device:GPU:0"):
    testNN = NeuralNetwork(name="main")
    
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
    saver.restore(sess, save_file)
    
    